In [0]:
################### STG automation ####################

## To be deleted
# Need to dynamise it. This is a PoC. 
# Wow.. It's working (party) (party)
# One notebook to rule them all.. all the way down

path = '/Volumes/odp-aws-dls3-eu-central-1-a-sources/slsi_adb/raw_databricks_stg/IN/SAP-P3B_100_FL4137-ZCOL-ZCIC-Complaints_FTP_FULL_ALL_20241010-003037_RAW_20241010-062508.csv'

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_date
from pyspark.sql.types import StringType
import json
import traceback

spark = SparkSession.builder.appName("DynamicColumnMapping").getOrCreate()

In [0]:
# This is a Sample for Config file

# [
#   {
#     "table_name": "table1",
#     "csv_path": "/mnt/volume/data_table1.csv",
#     "mapping_path": "/mnt/volume/mapping_table1.json"
#   },
#   {
#     "table_name": "table2",
#     "csv_path": "/mnt/volume/data_table2.csv",
#     "mapping_path": "/mnt/volume/mapping_table2.json"
#   }
# ]

In [0]:
# Column Mapping file 
# [
#   {
#     "csv_column": "csv_col1",
#     "table_column": "table_col1"
#   },
#   {
#     "csv_column": "csv_col2",
#     "table_column": "table_col2"
#   },
#   {
#     "csv_column": "csv_col3",
#     "table_column": "table_col3"
#   }
# ]

In [0]:
# Load configuration file
config_file_path = "/mnt/volume/config.json"  # Path to the config file
with open(config_file_path, "r") as file:
    config_data = json.load(file)

In [0]:
for table_config in config_data:
    table_name = table_config["table_name"]
    csv_path = table_config["csv_path"]
    mapping_path = table_config["mapping_path"]

    try:
        print(f"Processing table: {table_name}")

        # Load column mapping from JSON file
        mapping_df = spark.read.json(mapping_path)
        column_mapping = {row["csv_column"]: row["table_column"] for row in mapping_df.collect()}

        ## To fetch the latest file
        files = dbutils.fs.ls(csv_path)
 
        # Sort the files by modification time in descending order (most recent first)
        latest_file = sorted(files, key=lambda f: f.modificationTime, reverse=True)[0]
        csv_file_path = latest_file.path

        print(csv_file_path)

        # Load CSV data to be transformed
        csv_df = spark.read.option("header", True).csv(csv_file_path)

        # Dynamically rename columns based on the mapping
        renamed_df = csv_df.select([col(c).alias(column_mapping[c]) if c in column_mapping else col(c) for c in csv_df.columns])

        # Convert all column data types to StringType
        string_df = renamed_df.select([col(c).cast(StringType()).alias(c) for c in renamed_df.columns])

        # Add export_date column with the current date
        final_df = string_df.withColumn("export_date", current_date().cast(StringType()))

        # Show transformed DataFrame
        final_df.show()

        # Truncate the table before loading data
        spark.sql(f"TRUNCATE TABLE {table_name}")

        # Write the DataFrame to the table
        final_df.write.format("delta").mode("append").saveAsTable(table_name)

        print(f"Completed processing for table: {table_name}")

    except Exception as e:
        # Log the error and continue with the next table
        print(f"Error processing table: {table_name}")
        print(f"Error details: {e}")
        print(traceback.format_exc())
        # continue  # Proceed to the next table